### Calculate error statistics from population model (more memory efficient)

We require bilby to load the population hyperposterior samples and gwpopulation to create the population model

In [ ]:
import gwpopulation
gwpopulation.set_backend('jax')
import h5py
from bilby.core.result import read_in_result

# load in population-error package
from population_error import error_statistics

Load in injection set and posterior set

In [2]:
# dictionary of injections, containing arrays of shape (Ninj). Must contain 'prior' key and should also contain 'total_generated'
# adapted from publicly available GW injections at https://zenodo.org/records/7890398
with h5py.File('data/GWTC3_injections.h5') as f:
    injections = {k: f[k][()] for k in f.keys()}
    
# dictionary of gw samples, containing arrays of shape (Nobs, NPE). Must contain 'prior' key
# adapted from publicly available GW posteriors at https://zenodo.org/records/6513631 and https://zenodo.org/records/8177023
with h5py.File('data/GWTC3_posteriors.h5') as f:
    event_posteriors = {k: f[k][()] for k in f.keys()}

Create gwpopulation model

In [3]:
model_list = [
    gwpopulation.models.mass.SinglePeakSmoothedMassDistribution(), 
    gwpopulation.models.spin.iid_spin,
    gwpopulation.models.redshift.PowerLawRedshift()
    ]

model_function = gwpopulation.experimental.jax.NonCachingModel(model_list)

Load in hyperposterior samples

In [4]:
# copied from GWTC-3 data release at https://zenodo.org/records/11254021 (analyses_PowerLawPeak.tar.gz)
gwtc3_result = read_in_result('data/o1o2o3_mass_c_iid_mag_iid_tilt_powerlaw_redshift_result.json')
hyperposterior = gwtc3_result.posterior

Calculate error statistics

In [5]:
statistics = error_statistics(
    model_function, 
    injections, 
    event_posteriors, 
    hyperposterior, 
    include_likelihood_correction=True, 
    conversion_function=gwpopulation.conversions.convert_to_beta_parameters
    )

Nobs not provided, assuming Nobs = 69


Computing single event covariance weights integrated over hyperposterior samples: 100%|██████████| 11469/11469 [00:22<00:00, 504.68it/s]
Computing selection covariance weights integrated over hyperposterior samples: 100%|██████████| 11469/11469 [00:11<00:00, 1040.01it/s]
For each posterior sample, average single event covariance with another posterior sample: 100%|██████████| 11469/11469 [00:11<00:00, 1004.06it/s]
For each posterior sample, average selection covariance with another posterior sample: 100%|██████████| 11469/11469 [00:11<00:00, 1041.89it/s]



Your inference loses approximately 0.263 bits of information to Monte Carlo approximations.
Of the total information loss
 * 0.255 bits is from uncertainty in the posterior. Of this
    * 11.0% is from the single-event Monte Carlo integration
    * 89.0% is from the selection Monte Carlo integration
 * 0.00827 bits is from bias in the posterior. Of the total bias
    * 3.0% is from the single-event Monte Carlo integration
    * 101.3% is from the selection Monte Carlo integration
    * -4.3% is from correlations in the uncertainty of the single-event and selection MC integrals


Note the uncertainty dominates the total information loss, which is generically true. There is a small additional information loss in the posterior due to bias in the estimate of the likelihood and posterior probability density. In GWTC-3, the systematic uncertainty was mostly due to the selection function estimate.

Note the contribution to the bias from the correlations in the uncertainty of the single-event and selection MC integrals is negative. This is not a bug! Since the bias the a directional quantity, the overall information loss will be positive, but the contributions to the total bias can be in different directions, meaning they will have a relative minus sign.